<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modularizando-Código" data-toc-modified-id="Modularizando-Código-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modularizando Código</a></span></li></ul></div>

In [1]:
!pip install requests beautifulsoup4 lxml

Links:
* https://www.pluralsight.com/guides/extracting-data-html-beautifulsoup
* https://stackoverflow.com/questions/41982475/scraper-in-python-gives-access-denied
* https://stackoverflow.com/questions/24962039/python-urllib-getting-access-denied-when-browser-works/24962277

In [1]:
# Importando bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime

# Definindo parâmetros de pesquisa
BASIC_URL = 'https://www.fundamentus.com.br/detalhes.php?papel='
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
CUSTOM_HEADINGS = ['Dia', 'Mês', '30 dias', '12 meses'] + [str(datetime.now().year - i) for i in range(6)]
COLUMNS = ['Papel', 'Empresa', 'Setor', 'Subsetor', 'Cotação', 'Min 52 sem', 'Max 52 sem'] + CUSTOM_HEADINGS[:7] + \
          ['P/L', 'P/VP', 'LPA', 'VPA', 'EV / EBITDA', 'ROE', 'ROIC', 'Marg. Bruta', 'Marg. EBIT', 'Marg. Líquida', 'Lucro Líquido']

ATIVO = 'ITSA4'
url = BASIC_URL + ATIVO

# Coletando e transformando conteúdo em HTML com BeautifulSoup
html_content = requests.get(url, headers=HEADERS)
soup = BeautifulSoup(html_content.text)

# Encontrando tabelas de classe específica com indicadores financeiros do ativo
invest_table = soup.find_all('table', attrs={'class': 'w728'})

# Extraindo dados para todas as tabelas de análise
financial_tables = []
for table in invest_table:
    table_data = table.find_all('tr')
    # Iterando sobre as tabelas internas
    for td_data in table_data:
        
        # Extraindo tabelas e listas específicas de colunas e valores
        td = td_data.find_all('td')
        headings = [h.text.replace('?', '').strip() for h in td if (('?' in h.text) or (h.text in CUSTOM_HEADINGS))]
        contents = [c.text.strip() for c in td if ('?' not in c.text) and (c.text not in headings)]
        
        # Gerando dicionário com os elementos e adicionando na lista geral de indicadores
        table_data = {k: v for k, v in zip(headings, contents)}
        if table_data != {}:
            financial_tables.append(table_data)
            
# Juntando em dicionário único e transformando em DataFrame
financial_data = dict(pair for dictionary in financial_tables for pair in dictionary.items())
df_financial = pd.DataFrame(financial_data, index=[0])
df_financial = df_financial.loc[:, COLUMNS]
df_financial

,Papel,Empresa,Setor,Subsetor,Cotação,Min 52 sem,Max 52 sem,Dia,Mês,30 dias,12 meses,2021,2020,2019,P/L,P/VP,LPA,VPA,EV / EBITDA,ROE,ROIC,Marg. Bruta,Marg. EBIT,Marg. Líquida,Lucro Líquido
0,ITSA4,ITAÚSA PN N1,Intermediários Financeiros,Bancos,"11,23","8,61","12,02","0,45%","0,82%","1,64%","18,21%","-2,89%","-12,74%","27,39%","8,46","1,55","1,33","7,27","50,07","18,3%","1,7%","34,0%","16,4%","161,8%",3.514.000.000


___

In [2]:
# Importando bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime

# Definindo parâmetros de pesquisa
BASIC_URL = 'https://www.fundamentus.com.br/detalhes.php?papel='
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
CUSTOM_HEADINGS = ['Dia', 'Mês', '30 dias', '12 meses'] + [str(datetime.now().year - i) for i in range(6)]

# Lista de ativos e indicadores financeiros
ATIVOS = ['BBSE3', 'ALUP11', 'SULA11', 'EGIE3', 'B3SA3', 'MOVI3', 'ABEV3', 'CPLE6', 'ITSA4', 'PNVL3', 'SAPR4']
COLUMNS = ['Papel', 'Empresa', 'Setor', 'Subsetor', 'Cotação', 'Min 52 sem', 'Max 52 sem'] + CUSTOM_HEADINGS[:7] + \
          ['P/L', 'P/VP', 'LPA', 'VPA', 'EV / EBITDA', 'ROE', 'ROIC', 'Marg. Bruta', 'Marg. EBIT', 
           'Marg. Líquida', 'Lucro Líquido']

# Criando DataFrame vazio e iterando sobre lista de ativos
df_financial = pd.DataFrame()
for ativo in ATIVOS:
    # Criando url de requisição do ativo
    url = BASIC_URL + ativo

    # Coletando e transformando conteúdo em HTML com BeautifulSoup
    html_content = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(html_content.text)

    # Encontrando tabelas de classe específica com indicadores financeiros do ativo
    invest_table = soup.find_all('table', attrs={'class': 'w728'})

    # Extraindo dados para todas as tabelas de análise
    financial_tables = []
    for table in invest_table:
        table_data = table.find_all('tr')
        # Iterando sobre as tabelas internas
        for td_data in table_data:

            # Extraindo tabelas e listas específicas de colunas e valores
            td = td_data.find_all('td')
            headings = [h.text.replace('?', '').strip() for h in td if (('?' in h.text) or (h.text in CUSTOM_HEADINGS))]
            contents = [c.text.strip() for c in td if ('?' not in c.text) and (c.text not in headings)]

            # Gerando dicionário com os elementos e adicionando na lista geral de indicadores
            table_data = {k: v for k, v in zip(headings, contents)}
            if table_data != {}:
                financial_tables.append(table_data)

    # Juntando em dicionário único e transformando em DataFrame
    financial_data = dict(pair for dictionary in financial_tables for pair in dictionary.items())
    df_ativo = pd.DataFrame(financial_data, index=[0])
    df_ativo = df_ativo.loc[:, COLUMNS]
    
    # Adicionando ao DataFrame final
    df_financial = df_financial.append(df_ativo)
df_financial

,Papel,Empresa,Setor,Subsetor,Cotação,Min 52 sem,Max 52 sem,Dia,Mês,30 dias,12 meses,2021,2020,2019,P/L,P/VP,LPA,VPA,EV / EBITDA,ROE,ROIC,Marg. Bruta,Marg. EBIT,Marg. Líquida,Lucro Líquido
0,BBSE3,BBSEGURIDADE ON NM,Previdência e Seguros,Seguradoras,"19,50","19,50","28,98","-1,81%","-6,50%","-9,18%","-23,54%","-31,42%","-11,25%","43,62%","10,42","5,65","1,87","3,45",-,"54,3%",-,-,-,"0,0%",753.702.000
0,ALUP11,ALUPAR UNT N2,Energia Elétrica,Energia Elétrica,"24,73","22,09","28,15","-1,00%","0,73%","-1,12%","5,91%","-5,74%","1,14%","53,50%","5,38","1,14","4,60","21,75","3,21","21,1%","20,4%","67,8%","66,1%","41,2%",332.345.000
0,SULA11,Sul America UNT N2,Previdência e Seguros,Seguradoras,"29,32","28,20","42,41","-1,11%","-2,04%","-7,62%","-29,44%","-29,67%","-21,37%","115,16%","6,74","1,55","4,35","18,87","-3,99","23,1%","4,5%","10,2%","2,6%","9,7%",29.339.000
0,EGIE3,ENGIE BRASIL ON,Energia Elétrica,Energia Elétrica,"38,31","36,16","43,90","-0,49%","3,63%","0,79%","-4,35%","-8,84%","-10,19%","59,20%","13,20","3,83","2,90","10,01","6,80","29,0%","17,6%","44,4%","42,2%","17,7%",318.788.000
0,B3SA3,B3 ON,Serviços Financeiros Diversos,Serviços Financeiros Diversos,"13,65","13,54","20,47","-3,87%","-10,49%","-16,51%","-29,07%","-31,15%","49,95%","63,26%","17,85","3,68","0,76","3,70","9,83","20,6%","21,3%","90,9%","63,9%","45,3%",1.193.340.000
0,MOVI3,MOVIDA ON NM,Diversos,Aluguel de carros,"18,74","15,61","22,86","-0,79%","-16,38%","-11,35%","14,90%","-8,00%","9,17%","125,74%","13,47","2,65","1,39","7,06","8,75","19,7%","11,8%","33,2%","21,3%","12,5%",173.921.000
0,ABEV3,AMBEV S/A ON,Bebidas,Cervejas e Refrigerantes,"17,36","11,71","19,60","-1,03%","4,33%","3,64%","37,65%","11,45%","-14,20%","24,08%","18,75","3,46","0,93","5,01","12,86","18,5%","16,7%","52,6%","23,1%","22,4%",2.885.780.000
0,CPLE6,COPEL PNB,Energia Elétrica,Energia Elétrica,"6,64","5,32","6,81","-1,04%","8,50%","9,75%","21,49%","-2,46%","12,38%","135,32%","5,16","0,90","1,29","7,37","5,25","17,4%","9,3%","24,8%","20,1%","17,8%",949.988.000
0,ITSA4,ITAÚSA PN N1,Intermediários Financeiros,Bancos,"11,23","8,61","12,02","0,45%","0,82%","1,64%","18,21%","-2,89%","-12,74%","27,39%","8,46","1,55","1,33","7,27","50,07","18,3%","1,7%","34,0%","16,4%","161,8%",3.514.000.000
0,PNVL3,PANVEL FARMÁCIAS ON,Comércio e Distribuição,Medicamentos e Outros Produtos,"17,86","17,13","26,40","-3,33%","-6,80%","-10,53%","-21,61%","-17,55%","11,14%","91,72%","34,08","2,62","0,52","6,81","14,78","7,7%","4,6%","30,2%","2,6%","2,6%",20.295.000


___

# Modularizando Código

In [3]:
class RaioXAtivos:
    """
    Classe responsável por propor análises detalhadas sobre
    ativos financeiros a partir de consulta a uma URL do
    site FUNDAMENTUS, permitindo assim a construção de uma
    série de métodos capazes de receber uma lista de ações
    da bolsa de valores e extrair uma série de indicadores
    financeiros dos ativos selecionados, como por exemplo, 
    as cotações atualizadas, variações de época, lucros, 
    margens, P/L, P/VP, entre outros.
    
    Com essa classe, o usuário poderá aprimorar o 
    gerenciamento de investimentos financeiros realizados
    na bolsa de valores brasileria através de uma análise
    específica em uma série de ativos.
    
    Atributos da classe
    -------------------
    :attr host:
        Host utilizado para conexão ao banco de dados. Caso
        o banco esteja hospedado localmente, é possível 
        passar o ip local da máquina ou a string 'localhost'
        [type: string]
        
    :attr database:
        Informação do banco de dados a ser utilizado na
        conexão.
        [type: string]
        
    :attr user:
        Usuário do banco de dados utilizado para formalizar
        a conexão.
        [type: string]
        
    :attr password:
        Senha do usuário de acesso ao banco de dados.
        [type: string]
        
    :attr verbose:
        Atributo de gerenciamento de loggers durante a
        execução de algumas operações da classe.
        [type: bool, default=True]
    """
    
    def __init__(self, 
                 basic_url='https://www.fundamentus.com.br/detalhes.php?papel=',
                 headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'},
                 custom_headings=['Dia', 'Mês', '30 dias', '12 meses'] + [str(datetime.now().year-i) for i in range(6)],
                 columns=['Papel', 'Empresa', 'Setor', 'Subsetor', 'Cotação', 'Data últ cot', 'Min 52 sem', 'Max 52 sem'] + \
                         CUSTOM_HEADINGS[:7] + ['P/L', 'P/VP', 'LPA', 'VPA', 'EV / EBITDA', 'Div. Yield', 'ROE', 'ROIC', 
                                               'Marg. Bruta', 'Marg. EBIT', 'Marg. Líquida', 'Lucro Líquido']):
        self.basic_url = basic_url
        self.headers = headers
        self.custom_headings = custom_headings
        self.columns = columns
        
    def analise_financeira(self, ativos, save=False, filepath='indicadores_financeiros.csv'):
        
        # Iterando sobre lista de ativos da classe
        df_ind_financeiros = pd.DataFrame()
        for ativo in ativos:
            # Criando url de requisição do ativo
            url = self.basic_url + ativo

            # Coletando e transformando conteúdo em HTML com BeautifulSoup
            html_content = requests.get(url, headers=HEADERS)
            soup = BeautifulSoup(html_content.text)

            # Encontrando tabelas de classe específica com indicadores financeiros do ativo
            invest_table = soup.find_all('table', attrs={'class': 'w728'})

            # Extraindo dados para todas as tabelas de análise
            financial_tables = []
            for table in invest_table:
                table_data = table.find_all('tr')
                # Iterando sobre as tabelas internas
                for td_data in table_data:

                    # Extraindo tabelas e listas específicas de colunas e valores
                    td = td_data.find_all('td')
                    headings = [h.text.replace('?', '').strip() for h in td if (('?' in h.text) or (h.text in CUSTOM_HEADINGS))]
                    contents = [c.text.strip() for c in td if ('?' not in c.text) and (c.text not in headings)]

                    # Gerando dicionário com os elementos e adicionando na lista geral de indicadores
                    table_data = {k: v for k, v in zip(headings, contents)}
                    if table_data != {}:
                        financial_tables.append(table_data)

            # Juntando em dicionário único e transformando em DataFrame
            financial_data = dict(pair for dictionary in financial_tables for pair in dictionary.items())
            df_ativo = pd.DataFrame(financial_data, index=[0])
            try:
                df_ativo = df_ativo.loc[:, self.columns]
            except KeyError as ke:
                continue
                
            # Adicionando ao DataFrame final
            df_ind_financeiros = df_ind_financeiros.append(df_ativo)
            
        # Verificando salvamento do arquivo
        if save:
            df_ind_financeiros.to_csv(filename, index=False)
        
        return df_ind_financeiros

In [4]:
# Instanciando classe e gerando análise financeira para um grupo de ativos
rx_ativos = RaioXAtivos()
df_ativos = rx_ativos.analise_financeira(ativos=ATIVOS)
df_ativos

,Papel,Empresa,Setor,Subsetor,Cotação,Data últ cot,Min 52 sem,Max 52 sem,Dia,Mês,30 dias,12 meses,2021,2020,2019,P/L,P/VP,LPA,VPA,EV / EBITDA,Div. Yield,ROE,ROIC,Marg. Bruta,Marg. EBIT,Marg. Líquida,Lucro Líquido
0,BBSE3,BBSEGURIDADE ON NM,Previdência e Seguros,Seguradoras,"19,50",23/08/2021,"19,50","28,98","-1,81%","-6,50%","-9,18%","-23,54%","-31,42%","-11,25%","43,62%","10,42","5,65","1,87","3,45",-,"5,1%","54,3%",-,-,-,"0,0%",753.702.000
0,ALUP11,ALUPAR UNT N2,Energia Elétrica,Energia Elétrica,"24,73",23/08/2021,"22,09","28,15","-1,00%","0,73%","-1,12%","5,91%","-5,74%","1,14%","53,50%","5,38","1,14","4,60","21,75","3,21","3,4%","21,1%","20,4%","67,8%","66,1%","41,2%",332.345.000
0,SULA11,Sul America UNT N2,Previdência e Seguros,Seguradoras,"29,32",23/08/2021,"28,20","42,41","-1,11%","-2,04%","-7,62%","-29,44%","-29,67%","-21,37%","115,16%","6,74","1,55","4,35","18,87","-3,99","4,0%","23,1%","4,5%","10,2%","2,6%","9,7%",29.339.000
0,EGIE3,ENGIE BRASIL ON,Energia Elétrica,Energia Elétrica,"38,31",23/08/2021,"36,16","43,90","-0,49%","3,63%","0,79%","-4,35%","-8,84%","-10,19%","59,20%","13,20","3,83","2,90","10,01","6,80","6,8%","29,0%","17,6%","44,4%","42,2%","17,7%",318.788.000
0,B3SA3,B3 ON,Serviços Financeiros Diversos,Serviços Financeiros Diversos,"13,65",23/08/2021,"13,54","20,47","-3,87%","-10,49%","-16,51%","-29,07%","-31,15%","49,95%","63,26%","17,85","3,68","0,76","3,70","9,83","6,5%","20,6%","21,3%","90,9%","63,9%","45,3%",1.193.340.000
0,MOVI3,MOVIDA ON NM,Diversos,Aluguel de carros,"18,74",23/08/2021,"15,61","22,86","-0,79%","-16,38%","-11,35%","14,90%","-8,00%","9,17%","125,74%","13,47","2,65","1,39","7,06","8,75","1,4%","19,7%","11,8%","33,2%","21,3%","12,5%",173.921.000
0,ABEV3,AMBEV S/A ON,Bebidas,Cervejas e Refrigerantes,"17,36",23/08/2021,"11,71","19,60","-1,03%","4,33%","3,64%","37,65%","11,45%","-14,20%","24,08%","18,75","3,46","0,93","5,01","12,86","2,8%","18,5%","16,7%","52,6%","23,1%","22,4%",2.885.780.000
0,CPLE6,COPEL PNB,Energia Elétrica,Energia Elétrica,"6,64",23/08/2021,"5,32","6,81","-1,04%","8,50%","9,75%","21,49%","-2,46%","12,38%","135,32%","5,16","0,90","1,29","7,37","5,25","14,6%","17,4%","9,3%","24,8%","20,1%","17,8%",949.988.000
0,ITSA4,ITAÚSA PN N1,Intermediários Financeiros,Bancos,"11,23",23/08/2021,"8,61","12,02","0,45%","0,82%","1,64%","18,21%","-2,89%","-12,74%","27,39%","8,46","1,55","1,33","7,27","50,07","2,8%","18,3%","1,7%","34,0%","16,4%","161,8%",3.514.000.000
0,PNVL3,PANVEL FARMÁCIAS ON,Comércio e Distribuição,Medicamentos e Outros Produtos,"17,86",23/08/2021,"17,13","26,40","-3,33%","-6,80%","-10,53%","-21,61%","-17,55%","11,14%","91,72%","34,08","2,62","0,52","6,81","14,78","1,0%","7,7%","4,6%","30,2%","2,6%","2,6%",20.295.000


In [105]:
# Lendo arquivo com todos os ativos da bolsa e gerando uma base completa
siglas_ativos = pd.read_csv(os.path.join(DATA_PATH, 'acoes_bolsa.txt'))
ativos_bolsa = siglas_ativos['SIGLA'].values
df_ativos_completo = rx_ativos.analise_financeira(ativos=ativos_bolsa)
print(f'Dimensões da base gerada: {df_ativos_completo.shape}')
df_ativos_completo.head()

Dimensões da base gerada: (262, 27)


,Papel,Empresa,Setor,Subsetor,Cotação,Data últ cot,Min 52 sem,Max 52 sem,Dia,Mês,30 dias,12 meses,2021,2020,2019,P/L,P/VP,LPA,VPA,EV / EBITDA,Div. Yield,ROE,ROIC,Marg. Bruta,Marg. EBIT,Marg. Líquida,Lucro Líquido
0,RRRP3,3R PETROLEUM ON NM,"Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição","34,93",20/08/2021,"20,69","49,00","3,34%","-9,76%","-16,83%","68,02%","-6,10%","78,94%","0,00%","0,00","2,54","0,00","13,74",-,"0,0%",-,-,-,-,-,24.107.000
0,TTEN3,3TENTOS ON NM,Agropecuária,Agricultura,"10,45",20/08/2021,"10,18","12,15","2,45%","-7,52%","-11,44%","-13,64%","-13,64%","0,00%","0,00%","0,00","6,02","0,00","1,73",-,"0,0%",-,-,-,-,-,114.276.000
0,ABCB4,ABC Brasil PN N2,Intermediários Financeiros,Bancos,"15,10",20/08/2021,"11,10","17,60","0,47%","1,34%","0,00%","17,07%","1,11%","-20,07%","26,82%","7,76","0,77","1,95","19,72",-,"5,0%","9,9%",-,-,-,"0,0%",138.140.000
0,AERI3,AERIS ON NM,Máquinas e Equipamentos,Máq. e Equip. Industriais,"8,19",20/08/2021,"6,15","13,13","2,38%","-6,40%","-11,75%","24,89%","-18,10%","52,49%","0,00%","0,00","6,23","0,00","1,31",-,"0,0%",-,-,-,-,-,18.723.000
0,AESB3,AES BRASIL ON NM,Energia Elétrica,Energia Elétrica,"14,58",20/08/2021,"13,16","17,40","4,22%","5,60%","1,92%","-12,39%","-12,39%","0,00%","0,00%","250,85","3,24","0,06","4,50","82,86","1,6%","1,3%","1,4%","32,6%","22,7%","4,1%",23.197.000


In [107]:
df_ativos_completo.to_csv('ativos.csv')

In [6]:
url = 'https://www.infomoney.com.br/cotacoes/empresas-b3/'
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html lang="pt-BR">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <style>
   @charset "UTF-8";:root{--sapphire:#0091FF;--apatite:#32C5FF;--ruby:#FF5252;--xpamber:#F7B500;--gray:#6c757d;--amazonite:#44D7B6;--lightgray:#F5F7F8;--darkgray:#212121;--gray:#6C757D;--gray24:rgba(108, 117, 125, 0.24);--blue:#0091FF;--lightblue:#32C5FF;--yellow:#ffc709;--red:#FF5252;--emerald:#1eb980}html{scroll-behavior:smooth}body{font-family:'Helvetica Neue',sans-serif;color:#000}a{color:var(--sapphire)}.hl-hat{display:block;font-size:12px;line-height:16px;font-weight:600;margin-bottom:4px;color:#666;width:100%}.hl-title{display:block}.hl-title a{color:#000}.hl-title-2{font-size:24px;line-height:28px;letter-spacing:-.5px;font-weight:700}.hl-title-2 a{color:#000}.hl-title-4{font-size:20px;line-height:24px;letter-spacing:-.5px;font-weight:700}.hl-title-4 a{color:#111}.hl-title-8{font-size:14px;line-height:16px;font-

In [13]:
listagem_acoes = soup.find_all('table')
print(len(listagem_acoes))
listagem_acoes

11


[<table class="default-table">
 <thead>
 <tr>
 <th>Empresas</th>
 <th>Ativos</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td class="higher">VIA S.A</td>
 <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/via-viia3f/">VIIA3F</a>
 </td>
 </tr>
 <tr>
 <td class="higher">Armac</td>
 <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/armac-arml3/">ARML3</a>
 </td>
 </tr>
 <tr>
 <td class="higher">Multilaser</td>
 <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/multilaser-mlas3/">MLAS3</a>
 </td>
 </tr>
 <tr>
 <td class="higher">CBA</td>
 <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/cba-cbav3/">CBAV3</a>
 </td>
 </tr>
 <tr>
 <td class="higher">3tentos</td>
 <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/3tentos-tten3/">TTEN3</a>
 </td>
 </tr>
 <tr>
 <td class="higher">BR Partners</td>
 <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/brpartners-brbi11/">BRBI11</a>
 </td>
 </tr>
 <t

In [14]:
exemplo = listagem_acoes[0]
exemplo

<table class="default-table">
<thead>
<tr>
<th>Empresas</th>
<th>Ativos</th>
</tr>
</thead>
<tbody>
<tr>
<td class="higher">VIA S.A</td>
<td class="strong">
<a href="https://www.infomoney.com.br/cotacoes/via-viia3f/">VIIA3F</a>
</td>
</tr>
<tr>
<td class="higher">Armac</td>
<td class="strong">
<a href="https://www.infomoney.com.br/cotacoes/armac-arml3/">ARML3</a>
</td>
</tr>
<tr>
<td class="higher">Multilaser</td>
<td class="strong">
<a href="https://www.infomoney.com.br/cotacoes/multilaser-mlas3/">MLAS3</a>
</td>
</tr>
<tr>
<td class="higher">CBA</td>
<td class="strong">
<a href="https://www.infomoney.com.br/cotacoes/cba-cbav3/">CBAV3</a>
</td>
</tr>
<tr>
<td class="higher">3tentos</td>
<td class="strong">
<a href="https://www.infomoney.com.br/cotacoes/3tentos-tten3/">TTEN3</a>
</td>
</tr>
<tr>
<td class="higher">BR Partners</td>
<td class="strong">
<a href="https://www.infomoney.com.br/cotacoes/brpartners-brbi11/">BRBI11</a>
</td>
</tr>
<tr>
<td class="higher">GetNinjas</td>
<td clas

KeyError: 0

In [23]:
table_data = exemplo.find_all('td')
print(len(table_data))
table_data

197


[<td class="higher">VIA S.A</td>, <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/via-viia3f/">VIIA3F</a>
 </td>, <td class="higher">Armac</td>, <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/armac-arml3/">ARML3</a>
 </td>, <td class="higher">Multilaser</td>, <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/multilaser-mlas3/">MLAS3</a>
 </td>, <td class="higher">CBA</td>, <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/cba-cbav3/">CBAV3</a>
 </td>, <td class="higher">3tentos</td>, <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/3tentos-tten3/">TTEN3</a>
 </td>, <td class="higher">BR Partners</td>, <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/brpartners-brbi11/">BRBI11</a>
 </td>, <td class="higher">GetNinjas</td>, <td class="strong">
 <a href="https://www.infomoney.com.br/cotacoes/getninjas-ninj3/">NINJ3</a>
 </td>, <td class="higher">Dotz </td>, <td class="strong"

In [22]:
table_data.find('td')

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [24]:
for link in soup.find_all("a"):
    print("Inner Text: {}".format(link.text))
    print("Title: {}".format(link.get("title")))
    print("href: {}".format(link.get("href")))

Inner Text: 


Title: None
href: https://www.infomoney.com.br
Inner Text: Minha conta
Title: None
href: https://minhaconta.infomoney.com.br/
Inner Text: Minhas carteiras
Title: None
href: https://www.infomoney.com.br/ferramentas/carteira-de-acompanhamento
Inner Text: Sair
Title: None
href: https://auth.infomoney.com.br/api/auth/signout?redirect_to=https://www.infomoney.com.br%2Fwp-json%2Finfomoney%2Fv1%2Flogout-infomoney
Inner Text: 
account_circle
Entrar

Title: None
href: https://auth.infomoney.com.br/authentication/signin?response_type=code&client_id=aaeb000318befb9a2fe0f400&redirect_uri=http://ww3.infomoney.com.br/pages/authentication/callback&origin=portal-header&redirect_to=https://www.infomoney.com.br%2Fwp-json%2Finfomoney%2Fv1%2Flogin-infomoney
Inner Text: 
                                Expert XP                                
Title: Expert XP
href: https://eventoexpert.xpi.com.br/user/register?origin=infomoney&utm_source=infomoney&utm_medium=site&utm_campaign=HT
Inner Text:

Title: None
href: https://www.infomoney.com.br/cotacoes/shopify-inc-s2ho34/
Inner Text: C2ZR34
Title: None
href: https://www.infomoney.com.br/cotacoes/caesars-entt-c2zr34/
Inner Text: U2ST34
Title: None
href: https://www.infomoney.com.br/cotacoes/unity-softwr-u2st34/
Inner Text: S2EA34
Title: None
href: https://www.infomoney.com.br/cotacoes/sea-ltd-s2ea34/
Inner Text: P2EN34
Title: None
href: https://www.infomoney.com.br/cotacoes/penn-nationl-p2en34/
Inner Text: M2PW34
Title: None
href: https://www.infomoney.com.br/cotacoes/medical-p-tr-m2pw34/
Inner Text: K2CG34
Title: None
href: https://www.infomoney.com.br/cotacoes/kingsoft-chl-k2cg34/
Inner Text: D2KN34
Title: None
href: https://www.infomoney.com.br/cotacoes/draftkings-d2kn34/
Inner Text: C2ON34
Title: None
href: https://www.infomoney.com.br/cotacoes/cyrusone-inc-c2on34/
Inner Text: C2HD34
Title: None
href: https://www.infomoney.com.br/cotacoes/churchill-dw-c2hd34/
Inner Text: B2YN34
Title: None
href: https://www.infomoney.com.br/c

In [31]:
exemplo.find_all('a')[0].text

'VIIA3F'

___

In [51]:
url = 'https://www.infomoney.com.br/cotacoes/empresas-b3/'
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, 'lxml')

table_list = soup.find_all('table')

papeis = []
for categoria in table_list:
    
    for link in categoria.find_all('a'):
        papel = link.text.strip()

        if (papel != '') and (len(papel) <= 8 and len(papel) >= 5):
            if papel[-1] == 'F':
                papel = papel[:-1]

            papeis.append(papel)

In [126]:
url = 'https://www.fundamentus.com.br/fii_resultado.php'
html_content = requests.get(url, headers=HEADERS).text
soup = BeautifulSoup(html_content, 'lxml')
tables = soup.find_all('tr')

tickers = []
for row in tables[1:]:
    tickers.append(row.find_all('a')[0].text.strip())

In [127]:
len(tickers)

295

In [129]:
sorted(tickers)

['ABCP11',
 'AEFI11',
 'AFCR11',
 'AFHI11',
 'AFOF11',
 'AGCX11',
 'AIEC11',
 'ALMI11',
 'ALZR11',
 'ANCR11B',
 'ARCT11',
 'ARRI11',
 'ATSA11',
 'BARI11',
 'BBFI11B',
 'BBFO11',
 'BBPO11',
 'BBRC11',
 'BBVJ11',
 'BCFF11',
 'BCIA11',
 'BCRI11',
 'BICR11',
 'BLCP11',
 'BLMC11',
 'BLMG11',
 'BLMR11',
 'BMLC11',
 'BNFS11',
 'BPFF11',
 'BPML11',
 'BPRP11',
 'BRCO11',
 'BRCR11',
 'BREV11',
 'BRIM11',
 'BTAL11',
 'BTCR11',
 'BTLG11',
 'BTRA11',
 'BTWR11',
 'BVAR11',
 'CARE11',
 'CBOP11',
 'CCRF11',
 'CEOC11',
 'CNES11',
 'CORM11',
 'CPFF11',
 'CPTS11',
 'CRFF11',
 'CTXT11',
 'CVBI11',
 'CXCE11B',
 'CXCO11',
 'CXRI11',
 'CXTL11',
 'DEVA11',
 'DMAC11',
 'DOMC11',
 'DOVL11B',
 'DRIT11B',
 'EDFO11B',
 'EDGA11',
 'ELDO11B',
 'EQIN11',
 'ERCR11',
 'EURO11',
 'EVBI11',
 'FAED11',
 'FAMB11B',
 'FATN11',
 'FCFL11',
 'FEXC11',
 'FFCI11',
 'FIGS11',
 'FIIB11',
 'FIIP11B',
 'FISC11',
 'FIXX11',
 'FLCR11',
 'FLMA11',
 'FLRP11',
 'FMOF11',
 'FPAB11',
 'FPNG11',
 'FRBR11',
 'FTCE11B',
 'FVBI11',
 'FVPQ11',


In [125]:
tickers = [row.find_all('a')[0].text.strip() for row in soup.find_all('tr')[1:]]
tickers

['CSTB4',
 'MNSA3',
 'POPR4',
 'IVTT3',
 'CFLU4',
 'MNSA4',
 'PORP4',
 'CLAN3',
 'CSTB3',
 'PMET3',
 'GFSA3',
 'PLDN4',
 'DASA3',
 'MAPT4',
 'LCSA4',
 'WEST3',
 'MAPT3',
 'BHGR3',
 'SBFG3',
 'CBEE3',
 'RJCP3',
 'ABYA3',
 'PITI4',
 'AMER3',
 'TCNO3',
 'LCSA3',
 'MAGG3',
 'TCNO4',
 'LINX3',
 'BNCA3',
 'NORD3',
 'AMIL3',
 'BRTP3',
 'IDVL11',
 'ARTR3',
 'OMGE3',
 'HBSA3',
 'CVCB3',
 'PEFX3',
 'PEFX5',
 'TOYB4',
 'IMBI3',
 'CLAN4',
 'ELMD3',
 'TOYB3',
 'EBTP3',
 'EBTP4',
 'FBRA4',
 'BRTP4',
 'TELB3',
 'ITEC3',
 'IMBI4',
 'PRTX3',
 'DFVA3',
 'ASTA4',
 'VGOR4',
 'CRBM3',
 'HBRE3',
 'ORVR3',
 'OPCT3',
 'ECPR4',
 'BAHI4',
 'DFVA4',
 'ECPR3',
 'SMFT3',
 'TAMM4',
 'TAMM3',
 'EMBR3',
 'DTCY3',
 'ECOR3',
 'CRBM7',
 'WMBY3',
 'BAHI5',
 'MBLY3',
 'TDBH4',
 'TANC4',
 'CCIM3',
 'TELB4',
 'TDBH3',
 'FRIO3',
 'GBIO33',
 'TXRX3',
 'MEAL3',
 'LECO4',
 'SEDU3',
 'TESA3',
 'BIOM3',
 'IGUA3',
 'FGUI3',
 'IGUA6',
 'IGUA5',
 'PRVI3',
 'CPNY3',
 'TERI3',
 'VLID3',
 'AHEB6',
 'BIOM4',
 'MSPA3',
 'SEMP3',
 'SZPQ4'

In [108]:
tables[1].find_all('a')[0].text

'AALR3 '

In [100]:
tables.find_all('td')

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [94]:
soup

<html><body><p>Access denied</p></body></html>

In [73]:
exemplo = tabela_acoes.find_all('tr')[1]
exemplo.find_all('td')[1].text.strip()

'B3SA3'

In [90]:
papeis = []
for linhas_tabela in tabela_acoes.find_all('tr')[1:-1]:
    try:
        papeis.append(linhas_tabela.find_all('td')[1].text.strip())
    except:
        print(linhas_tabela)

In [92]:
len(papeis)

163

https://www.reddit.com/r/investimentos/comments/edbwca/onde_encontrar_todas_as_empresas_listadas_na/

In [ ]:
# Iterando sobre lista de ativos da classe
        df_ind_financeiros = pd.DataFrame()
        for ativo in ativos:
            # Criando url de requisição do ativo
            url = self.basic_url + ativo

            # Coletando e transformando conteúdo em HTML com BeautifulSoup
            html_content = requests.get(url, headers=HEADERS)
            soup = BeautifulSoup(html_content.text)

            # Encontrando tabelas de classe específica com indicadores financeiros do ativo
            invest_table = soup.find_all('table', attrs={'class': 'w728'})
            
for table in invest_table:
                table_data = table.find_all('tr')
                # Iterando sobre as tabelas internas
                for td_data in table_data:

                    # Extraindo tabelas e listas específicas de colunas e valores
                    td = td_data.find_all('td')
                    headings = [h.text.replace('?', '').strip() for h in td if (('?' in h.text) or (h.text in CUSTOM_HEADINGS))]
                    contents = [c.text.strip() for c in td if ('?' not in c.text) and (c.text not in headings)]

                    # Gerando dicionário com os elementos e adicionando na lista geral de indicadores
                    table_data = {k: v for k, v in zip(headings, contents)}
                    if table_data != {}:
                        financial_tables.append(table_data)

In [130]:
ativo

'SAPR4'

In [133]:
rx_ativos.basic_url

'https://www.fundamentus.com.br/detalhes.php?papel='

In [134]:
url = rx_ativos.basic_url + ativo
# Coletando e transformando conteúdo em HTML com BeautifulSoup
html_content = requests.get(url, headers=HEADERS)
headings = [h.text.replace('?', '').strip() for h in td_data.find_all('td') for td in table.find_all('tr') for table in soup.find_all('table', attrs={'class': 'w728'}) if (('?' in h.text) or (h.text in CUSTOM_HEADINGS))]

UnboundLocalError: local variable 'table' referenced before assignment

In [8]:
url = 'https://www.fundamentus.com.br/detalhes.php?papel=BTLG11'
html_content = requests.get(url, headers=HEADERS).text
soup = BeautifulSoup(html_content, 'lxml')
tables = soup.find_all('table')
tables

[<table class="w728">
 <tr>
 <td class="label w15"><span class="help tips" title="Código do FII">?</span><span class="txt">FII</span></td>
 <td class="data w35"><span class="txt">BTLG11</span></td>
 <td class="label destaque w2"><span class="help tips" title="Cotação de fechamento da ação no último pregão">?</span><span class="txt">Cotação</span></td>
 <td class="data destaque w3"><span class="txt">107,00</span></td>
 </tr>
 <tr>
 <td class="label"><span class="help tips" title="Nome do Fundo Imobiliário">?</span><span class="txt">Nome</span></td>
 <td class="data"><span class="txt">BTG PACTUAL LOGÍSTICA FUNDO DE INVESTIMENTO IMOBILIÁRIO</span></td>
 <td class="label"><span class="help tips" title="Data do último pregão em  que o ativo foi negociado">?</span><span class="txt">Data últ cot</span></td>
 <td class="data"><span class="txt">23/08/2021</span></td>
 </tr>
 <tr>
 <td class="label"><span class="help tips" title="Mandato: Desenvolvimento para Renda / Desenvolvimento para Venda /

In [9]:
# Encontrando tabelas de classe específica com indicadores financeiros do ativo
invest_table = soup.find_all('table', attrs={'class': 'w728'})
financial_tables = []
for table in invest_table:
    table_data = table.find_all('tr')
    # Iterando sobre as tabelas internas
    for td_data in table_data:

        # Extraindo tabelas e listas específicas de colunas e valores
        td = td_data.find_all('td')
        headings = [h.text.replace('?', '').strip() for h in td if (('?' in h.text) or (h.text in CUSTOM_HEADINGS))]
        contents = [c.text.strip() for c in td if ('?' not in c.text) and (c.text not in headings)]

        # Gerando dicionário com os elementos e adicionando na lista geral de indicadores
        table_data = {k: v for k, v in zip(headings, contents)}
        if table_data != {}:
            financial_tables.append(table_data)

In [10]:
financial_tables

[{'Cotação': '107,00', 'FII': 'BTLG11'},
 {'Data últ cot': '23/08/2021',
  'Nome': 'BTG PACTUAL LOGÍSTICA FUNDO DE INVESTIMENTO IMOBILIÁRIO'},
 {'Mandato': 'Renda', 'Min 52 sem': '96,49'},
 {'Max 52 sem': '113,10', 'Segmento': 'Híbrido'},
 {'Gestão': 'Ativa', 'Vol $ méd (2m)': '5.724.080'},
 {'Nro. Cotas': '15.228.960', 'Valor de mercado': '1.629.500.000'},
 {'Relatório': '30/06/2021', 'Últ Info Trimestral': '30/06/2021'},
 {'Dia': '-1,29%', 'FFO Yield': '4,35%', 'FFO/Cota': '4,66'},
 {'Div. Yield': '6,8%', 'Dividendo/cota': '6,54', 'Mês': '-2,46%'},
 {'30 dias': '-3,75%', 'P/VP': '1,05', 'VP/Cota': '101,75'},
 {'12 meses': '8,68%'},
 {'2021': '1,64%'},
 {'2020': '6,83%', 'Receita': '25.589.000'},
 {'2019': '18,79%', 'Venda de ativos': '43.770.900'},
 {'2018': '0,00%', 'FFO': '21.496.300'},
 {'2017': '0,00%', 'Rend. Distribuído': '32.590.000'},
 {'2016': '0,00%'},
 {'Ativos': '', 'Patrim Líquido': ''},
 {'Cap Rate': '3,1%', 'Qtd imóveis': '10', 'Área (m2)': '232.647'},
 {'Aluguel/m2': 

In [12]:
financial_data = dict(pair for dictionary in financial_tables for pair in dictionary.items())
df_ativo = pd.DataFrame(financial_data, index=[0])
df_ativo

,FII,Cotação,Nome,Data últ cot,Mandato,Min 52 sem,Segmento,Max 52 sem,Gestão,Vol $ méd (2m),Valor de mercado,Nro. Cotas,Relatório,Últ Info Trimestral,Dia,FFO Yield,FFO/Cota,Mês,Div. Yield,Dividendo/cota,30 dias,P/VP,VP/Cota,12 meses,2021,2020,Receita,2019,Venda de ativos,2018,FFO,2017,Rend. Distribuído,2016,Ativos,Patrim Líquido,Qtd imóveis,Área (m2),Cap Rate,Qtd Unidades,Aluguel/m2,Vacância Média,Imóveis/PL do FII,Preço do m2
0,BTLG11,"107,00",BTG PACTUAL LOGÍSTICA FUNDO DE INVESTIMENTO IM...,23/08/2021,Renda,"96,49",Híbrido,"113,10",Ativa,5.724.080,1.629.500.000,15.228.960,30/06/2021,30/06/2021,"-1,29%","4,35%","4,66","-2,46%","6,8%","6,54","-3,75%","1,05","101,75","8,68%","1,64%","6,83%",25.589.000,"18,79%",43.770.900,"0,00%",21.496.300,"0,00%",32.590.000,"0,00%",,,10,232.647,"3,1%",18,166,"2,3%","76,0%",5.322


In [37]:
cols_filter_fiis = ['FII', 'Nome', 'Segmento', 'Mandato', 'Cotação', 'Data últ cot', 'Min 52 sem', 'Max 52 sem'] + \
                    CUSTOM_HEADINGS[:7] + ['Div. Yield', 'Dividendo/cota', 'FFO Yield', 'FFO/Cota', 'P/VP', 'VP/Cota',
                                           'Receita', 'Venda de ativos', 'FFO', 'Rend. Distribuído', 'Ativos', 
                                           'Patrim Líquido', 'Qtd imóveis', 'Qtd Unidades', 'Imóveis/PL do FII',
                                           'Área (m2)', 'Aluguel/m2', 'Preço do m2', 'Cap Rate', 'Vacância Média']


In [38]:
df_ativo.loc[:, cols_filter_fiis]

,FII,Nome,Segmento,Mandato,Cotação,Data últ cot,Min 52 sem,Max 52 sem,Dia,Mês,30 dias,12 meses,2021,2020,2019,Div. Yield,Dividendo/cota,FFO Yield,FFO/Cota,P/VP,VP/Cota,Receita,Venda de ativos,FFO,Rend. Distribuído,Ativos,Patrim Líquido,Qtd imóveis,Qtd Unidades,Imóveis/PL do FII,Área (m2),Aluguel/m2,Preço do m2,Cap Rate,Vacância Média
0,BTLG11,BTG PACTUAL LOGÍSTICA FUNDO DE INVESTIMENTO IM...,Híbrido,Renda,"107,00",23/08/2021,"96,49","113,10","-1,29%","-2,46%","-3,75%","8,68%","1,64%","6,83%","18,79%","6,8%","6,54","4,35%","4,66","1,05","101,75",25.589.000,43.770.900,21.496.300,32.590.000,,,10,18,"76,0%",232.647,166,5.322,"3,1%","2,3%"


In [21]:
CUSTOM_HEADINGS

['Dia',
 'Mês',
 '30 dias',
 '12 meses',
 '2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016']

In [20]:
df_ativo.columns

Index(['FII', 'Cotação', 'Nome', 'Data últ cot', 'Mandato', 'Min 52 sem',
       'Segmento', 'Max 52 sem', 'Gestão', 'Vol $ méd (2m)',
       'Valor de mercado', 'Nro. Cotas', 'Relatório', 'Últ Info Trimestral',
       'Dia', 'FFO Yield', 'FFO/Cota', 'Mês', 'Div. Yield', 'Dividendo/cota',
       '30 dias', 'P/VP', 'VP/Cota', '12 meses', '2021', '2020', 'Receita',
       '2019', 'Venda de ativos', '2018', 'FFO', '2017', 'Rend. Distribuído',
       '2016', 'Ativos', 'Patrim Líquido', 'Qtd imóveis', 'Área (m2)',
       'Cap Rate', 'Qtd Unidades', 'Aluguel/m2', 'Vacância Média',
       'Imóveis/PL do FII', 'Preço do m2'],
      dtype='object')

In [18]:
len(cols_filter_fiis)

27

In [19]:
df_ativo.loc[:, cols_filter_fiis]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['Div.Yield'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [16]:
cols_filter_acoes = ['Papel', 'Empresa', 'Setor', 'Subsetor', 'Cotação', 'Data últ cot', 'Min 52 sem', 'Max 52 sem'] + \
                    CUSTOM_HEADINGS[:7] + ['P/L', 'P/VP', 'LPA', 'VPA', 'EV / EBITDA', 'Div. Yield', 'ROE', 'ROIC', 
                                           'Marg. Bruta', 'Marg. EBIT', 'Marg. Líquida', 'Lucro Líquido']
cols_filter_acoes

['Papel',
 'Empresa',
 'Setor',
 'Subsetor',
 'Cotação',
 'Data últ cot',
 'Min 52 sem',
 'Max 52 sem',
 'Dia',
 'Mês',
 '30 dias',
 '12 meses',
 '2021',
 '2020',
 '2019',
 'P/L',
 'P/VP',
 'LPA',
 'VPA',
 'EV / EBITDA',
 'Div. Yield',
 'ROE',
 'ROIC',
 'Marg. Bruta',
 'Marg. EBIT',
 'Marg. Líquida',
 'Lucro Líquido']

In [144]:
df_ind_financeiros = pd.DataFrame()
financial_data = dict(pair for dictionary in financial_tables for pair in dictionary.items())
df_ativo = pd.DataFrame(financial_data, index=[0])
try:
    df_ativo = df_ativo.loc[:, self.columns]
except KeyError as ke:
    pass

# Adicionando ao DataFrame final
df_ind_financeiros = df_ind_financeiros.append(df_ativo)

NameError: name 'self' is not defined

TODO: DEFINIR NOVAS COLUNAS PARA FIIS NA CLASSE